In [9]:
import pandas as pd
import spacy
from rapidfuzz import fuzz
from sklearn.model_selection import train_test_split

from data.dictionary import general_symptom_categories
from data.help_functions import clean_text, normalize_phrase, compute_scor_medical_diabet, infer_diagnosis_from_scor, \
    compute_scor_medical_cardio

# === 1. Inițializare ===
nlp = spacy.load("ro_core_news_sm")

# === 3. Încărcare și preprocesare text ===
df = pd.read_csv("processed/processed_data.csv", delimiter=';')
df.columns = df.columns.str.strip().str.replace('*', '', regex=False)

# Coloane de interes
col1 = "Ce alte simptome sau boli prezinți?"
col2 = "In prezent, care este cea mai mare provocare a ta? Ce crezi ca te împiedica sa slăbești si sa ai o stare buna de sănătate? "

# Curățare text
df[col1] = df[col1].fillna("").apply(clean_text)
df[col2] = df[col2].fillna("").apply(clean_text)

# Combinare + lematizare
df["text_lemmatized"] = (df[col1] + " " + df[col2]).apply(lambda x: normalize_phrase(x, nlp))

# === 5. Normalizează toate frazele din categorii ===
normalized_categories = {
    cat: [normalize_phrase(phrase, nlp) for phrase in phrases]
    for cat, phrases in general_symptom_categories.items()
}


# === 6. Funcție de etichetare fuzzy ===
def label_row(text_lemma, threshold=90):  # daca cuvantul indeplineste trashold atunci se aplica eticheta
    labels = set()
    for category, phrases in normalized_categories.items():
        for phrase in phrases:
            if fuzz.partial_ratio(phrase, text_lemma) >= threshold:
                labels.add(category)
                break
    return list(labels)


# === 7. Aplică etichetarea ===
df["labels"] = df["text_lemmatized"].apply(label_row)
df["scor_medical"] = df.apply(compute_scor_medical_diabet, axis=1)

# === 7.5. Suprascriere etichete pe baza scorului medical ===
df = df.apply(infer_diagnosis_from_scor, axis=1)


# === 7.7 Detectare afecțiuni și simptome cardiovasculare grave ===

def check_presence(text, phrase_list):
    tokens = text.split()
    for phrase in phrase_list:
        phrase_tokens = phrase.split()
        for i in range(len(tokens) - len(phrase_tokens) + 1):
            if tokens[i:i + len(phrase_tokens)] == phrase_tokens:
                return True
    return False


# 1. Expresii cu risc cardiovascular crescut (simptome, stil de viață etc.)
simptome_risc_cardio = [
    "durere piept", "presiune toracic", "nu pot respira", "sufocare",
    "tensiune mare", "hipertensiune", "puls mare", "batai rapide inima",
    "inima bate tare", "oboseala la efort", "edem", "picioare umflat",
    "retentie apa", "lesin", "vedere intunecat", "cap greu", "sforai puternic",
    "apnee somn", "colesterol mare", "trigliceride mari", "fumez zilnic",
    "stil viata sedentar", "nu fac miscare", "ma misc putin"
]

# Normalizează expresiile
simptome_risc_cardio_norm = [normalize_phrase(p, nlp) for p in simptome_risc_cardio]

# 2. Categoriile grave cardiovasculare (istoric sau diagnostic sever)
afectiuni_cardio_grave = {
    "infarct": [
        "infarct", "infarct miocardic", "atac de cord", "atac cardiac", "atac cord",
        "necroză miocardică", "ischemie miocardică", "angina instabilă", "angina pectorală",
        "stop cardiac", "infarct transmural", "infarct non-transmural",
        "ischemie coronariană", "boala coronariană", "sindrom coronarian acut"
    ],
    "avc": [
        "avc", "accident vascular cerebral", "accident vascular", "hemoragie cerebrală",
        "trombembolism cerebral", "ischemie cerebrală", "pareză", "hemipareză",
        "accident ischemic tranzitor", "tia", "accident vascular ischemic",
        "accident hemoragic", "hemoragie intracerebrală", "accident vascular lacunar",
        "embolism cerebral", "ischemie ischemică"
    ],
    "stent_sau_bypass": [
        "stent", "angioplastie", "bypass", "intervenție pe cord", "operație pe inimă",
        "revascularizare miocardică", "chirurgie coronariană", "by-pass coronarian",
        "angioplastie coronariană", "implantare stent coronarian", "intervenție coronariană percutană"
    ],
    "fibrilatie_sau_ritm": [
        "fibrilație atrială", "aritmie", "tulburări de ritm", "bătăi neregulate inimă",
        "palpitații severe", "tahicardie", "bradicardie", "extrasistole",
        "tulburări de conducere", "bloc atrioventricular", "fibrilație ventriculară",
        "flutter atrial", "tahiaritmie", "sindrom Wolff-Parkinson-White"
    ],
    "embolie_sau_tromboza": [
        "embolie", "embolie pulmonară", "tromboză", "cheag de sânge", "coagulare excesivă",
        "tromboembolism", "tromboflebită", "embolie arterială", "embolie venoasă profundă",
        "tromboză venoasă profundă", "tromboză arterială", "trombus", "trombocitopenie",
        "coagulopatie", "sindrom antifosfolipidic"
    ]
}

# Normalizează expresiile
afectiuni_cardio_grave_normalizate = {
    col: [normalize_phrase(p, nlp) for p in expresii]
    for col, expresii in afectiuni_cardio_grave.items()
}

# Creează coloane binare pentru fiecare afecțiune gravă cu funcția strictă
for col_name, phrase_list in afectiuni_cardio_grave_normalizate.items():
    df[col_name] = df["text_lemmatized"].apply(lambda text: int(check_presence(text, phrase_list)))

# Coloană finală: risc cardiovascular general
# Dacă are ORICE simptom de risc SAU ORICE afecțiune cardiovasculară gravă => risc_cardiovascular = 1
df["risc_cardiovascular"] = df["text_lemmatized"].apply(
    lambda text: int(check_presence(text, simptome_risc_cardio_norm)))

df["risc_cardiovascular"] = df.apply(
    lambda row: 1 if (
            row.get("infarct", 0) == 1 or
            row.get("avc", 0) == 1 or
            row.get("stent_sau_bypass", 0) == 1 or
            row.get("fibrilatie_sau_ritm", 0) == 1 or
            row.get("embolie_sau_tromboza", 0) == 1 or
            row.get("risc_cardiovascular", 0) == 1 or
            row.get("hipertensiune arteriala", 0) == 1 or
            row.get("dislipidemie (grăsimi crescute in sânge)", 0) == 1 or
            row.get("scor_medical_cardio", 0) > 25
    ) else 0,
    axis=1
)

# Calculează scorul medical cardio după funcția ta
df['scor_medical_cardio'] = df.apply(compute_scor_medical_cardio, axis=1)
# Asumând că ai deja coloana 'scor_medical_cardio' calculată în df
mediana_scor = df['scor_medical_cardio'].median()
print("Mediana scorului medical cardio este:", mediana_scor)

# === 8. Salvează dataseturile ===
train, tmp = train_test_split(df, test_size=0.4, random_state=42)
val, test = train_test_split(tmp, test_size=0.5, random_state=42)
train.to_csv('datasets/train/train.csv', sep=';', index=False)
val.to_csv('datasets/validation/val.csv', sep=';', index=False)
test.to_csv('datasets/test/test.csv', sep=';', index=False)

print("Etichete extrase:", sorted(set(label for labels in df["labels"] for label in labels)))

Mediana scorului medical cardio este: 16.0
Etichete extrase: ['cardio_vascular', 'gastro_hepato_renal', 'ginecologic_hormonal', 'inflamator_autoimun', 'metabolic_endocrin', 'neuro_psiho_energie']


In [21]:
train.head(3000)
y_test = train["risc_cardiovascular"].dtype
print(y_test)

int64


In [20]:
val.head(1000)
y_test = val["risc_cardiovascular"].dtype
print(y_test)

int64


In [19]:
test.head(1000)
y_test = test["risc_cardiovascular"].dtype
print(y_test)

int64
